In [1]:
import jax
import tax
import clu
import rlax
import tqdm
import haiku as hk
import numpy as np
import collections 
import jax.numpy as jnp
import matplotlib as mpl
import matplotlib.pyplot as plt

import typing
import optax
import chex
import tree
import mbrl
from jax import jit
from functools import partial
from torch.utils.data import DataLoader
from mbrl.common.nn import mlp_deterministic, mlp_multivariate_normal_diag
from mbrl.envs.oracle.pendulum import render, step, reset, env_params, angle_normalize, get_obs_pendulum
from mbrl.algs.rs import trajectory_search, forecast, score, plan

Environment = collections.namedtuple('Environment', ['step', 'reset']) 
tax.set_platform('cpu')

In [2]:
rng = jax.random.PRNGKey(10)
env = Environment(
    jit(lambda state, u: step(env_params, state, u)), 
    jit(reset)
)

action_size = 1
observation_size = 3
env_state, observation = env.reset(rng)

action = jnp.zeros((action_size))
env_state_next, true_Y, true_R, *_ = env.step(env_state, action)

In [3]:
def true_world(carry, t):
    keys, (env_state, observation), trajectory = carry
    action = trajectory[t]
    env_state_next, observation_next, reward, terminal, info = \
        env.step(env_state, action)
    carry = keys, (env_state_next, observation_next), trajectory
    return carry, {
        "observation": observation,
        "observation_next": observation_next,
        "reward": reward, "action": action, "terminal": 1 - terminal,
        "env_state": env_state, 'env_state_next': env_state_next
    }


# -- Random Data

buf = []
for _ in range(10):
    score = 0
    env_state, observation = env.reset(rng)
    for _ in range(200):
        rng, key = jax.random.split(rng)
        action = jax.random.uniform(key, (1,), minval=-2., maxval=2.)
        env_state, observation_next, reward, terminal, info = env.step(env_state, action)
        score += reward
        buf.append({
            'observation': observation,
            'observation_next': observation_next,
            'action': action,
            'reward': reward,
            'env_state': env_state,
            'env_state_next': env_state_next
        })
        observation = observation_next.copy()

    print(f'Random Score: {score}')
    
data = tax.reduce(buf)

Random Score: -1176.3677978515625
Random Score: -962.1798095703125
Random Score: -996.4786987304688
Random Score: -1703.94140625
Random Score: -833.6911010742188
Random Score: -1072.240966796875
Random Score: -1067.98828125
Random Score: -995.6116943359375
Random Score: -887.7622680664062
Random Score: -1183.324951171875


In [4]:
FLAGS_MODEL = 'D'

@chex.dataclass
class NormalizationState:
    observation_mean: jnp.ndarray
    observation_std: jnp.ndarray
    action_mean: jnp.ndarray
    action_std: jnp.ndarray

        
@chex.dataclass
class RState:
    params: typing.Any
    opt_state: typing.Any
    norm: NormalizationState


state_norm = NormalizationState(
    observation_mean = jnp.zeros((observation_size,)),
    observation_std = jnp.ones((observation_size,)),
    action_mean = jnp.zeros((action_size,)),
    action_std = jnp.ones((action_size,)),
)


if FLAGS_MODEL == 'D':
    rmodel = lambda x, a, y: mlp_deterministic(
        1, [32, 32], 
        final_tanh_activation=False)(jnp.concatenate([x, a, y], -1))
elif FLAGS_MODEL == 'P':
    rmodel = lambda x, a, y: mlp_multivariate_normal_diag(
        1, [32, 32], 
        use_tanh_bijector=False)(jnp.concatenate([x, a, y], -1))
else:
    raise NotImplementedError()
    
rmodel = hk.transform(rmodel)
rmodel = hk.without_apply_rng(rmodel)
rmodel_params = rmodel.init(rng, 
                            jnp.zeros((observation_size,)),
                            jnp.zeros((action_size,)),
                            jnp.zeros((observation_size,)),
                           )
rmodel_opt = optax.adabelief(learning_rate=5e-3)
rmodel_opt_state = rmodel_opt.init(rmodel_params)
rstate = RState(params=rmodel_params, opt_state=rmodel_opt_state, norm=state_norm)

if 'D' in FLAGS_MODEL:
    @partial(jit, static_argnums=(3, 4))
    def loss_fn(p, inputs, target, rmodel_def, type_loss: str = 'l1'):
        prediction = rmodel_def(p, *inputs)
        if type_loss == 'l1':
            loss = tax.l1_loss(prediction, target)
        loss = tax.l2_loss(prediction, target)
        loss = loss.mean()
        return loss
    
    
loss = jit(partial(loss_fn, rmodel_def=rmodel.apply, type_loss='l1'))

   
if 'P' in FLAGS_MODEL:
    @partial(jit, static_argnums=(3,))
    def loss_fn(p, inputs, target, rmodel_def):
        dist = rmodel_def(p, *inputs)
        loss = -dist.log_prob(target).mean()
        return loss

    loss = jit(partial(loss_fn, rmodel_def=rmodel.apply))
    
@partial(jit, static_argnums=(3, 4))
def update_fn(state, inputs, target, opt, loss_fn):
    l, g = jax.value_and_grad(loss_fn)(state.params, inputs, target)
    updates, opt_state = opt.update(g, state.opt_state)
    params = jax.tree_multimap(lambda p, u: p + u, state.params, updates)
    state = state.replace(params=params, opt_state=opt_state)
    metrics = {'loss': l}
    return state, metrics

update = jit(partial(update_fn, loss_fn=loss, opt=rmodel_opt))

In [5]:

        
def train_rmodel(
    data, state, loss_fn, update_fn,
    reward_model,
    seed: int = 42, batch_size:int = 256, 
    use_norm: bool = True, 
    max_epochs: int = 200, validation_size: float = 0.25,
    early_stopping_patience: int = 15, alpha_norm: float = 0.5,
):
    data = tree.map_structure(lambda v: np.array(v), data)

    # Assume that the initial normalization is (mean=0, std=1)
    # This setting correspond to no normalization.
    # If the flag `use_norm` is True, the function
    # will update the norm state based on the data
    # it received according a polyak rule (weighted update).
    if use_norm:
        observation_mean = data['observation'].mean(0)
        observation_std = data['observation'].std(0)
        action_mean = data['action'].mean(0)
        action_std = data['action'].std(0)
        new_observation_mean = (1-alpha_norm) * observation_mean + \
                             alpha_norm * state.norm.observation_mean
        new_observation_std = (1-alpha_norm) * observation_std + \
                             alpha_norm * state.norm.observation_std
        new_action_mean = (1-alpha_norm) * action_mean + \
                             alpha_norm * state.norm.action_mean        
        new_action_std = (1-alpha_norm) * action_std + \
                             alpha_norm * state.norm.action_std
    
        new_norm = NormalizationState(
            observation_mean=new_observation_mean,
            observation_std=new_observation_std,
            action_mean=new_action_mean,
            action_std=new_action_std
        )
        state = state.replace(norm=new_norm)
    
    def process(observation, action, observation_next):
        observation_next_norm = \
            (observation_next - state.norm.observation_mean) / (state.norm.observation_std + 1e-6)
        observation_norm = \
            (observation - state.norm.observation_mean) / (state.norm.observation_std + 1e-6)
        action_norm = (action - state.norm.action_mean) / (state.norm.action_std + 1e-6)
        inputs = (observation_norm, action_norm, observation_next_norm)
        return inputs

    ds = tax.DatasetDict(data)
    es = tax.EarlyStopping(patience=early_stopping_patience)
    ds_train, ds_valid = tax.random_splits(ds, validation_size)
    dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
    dl_valid = DataLoader(ds_valid, batch_size=batch_size)
    
    allinfo = []
    t = tqdm.trange(max_epochs)
    for range in t:
        store = tax.Store(decimals=4)
        for batch in dl_train:
            batch = tree.map_structure(lambda v: jnp.asarray(v), batch)
            action = batch['action']
            reward = batch['reward']
            observation = batch['observation']
            observation_next = batch['observation_next']
            inputs = process(observation, action, observation_next)
            state, info = update(state, inputs, reward)
            store.add(**{'loss/train': info['loss']})
        for batch in dl_valid:
            batch = tree.map_structure(lambda v: jnp.asarray(v), batch)
            action = batch['action']
            reward = batch['reward']
            observation = batch['observation']
            observation_next = batch['observation_next']
            inputs  = process(observation, action, observation_next)
            l  = loss(state.params, inputs, reward)
            store.add(**{'loss/valid': l})

        metrics = store.get()
        t.set_postfix(metrics)
        allinfo.append(metrics)
        validation_loss = metrics['loss/valid']
        if es.step(validation_loss):
            break

    # Sanity Check to automatically choose the right
    # inference model.
    dummy_prediction = reward_model(state.params, observation, action, observation_next)
    model = 'P'
    if isinstance(dummy_prediction, jnp.ndarray):
        model = 'D'
    
    # At this point, the training is done.
    # We return the state, the metrics
    # and the forward model inference ready 
    # to use according the training.
    @jit
    def rmodel_inference(rng, observation, action, observation_next):
        observation_norm = \
            (observation - state.norm.observation_mean) / (state.norm.observation_std + 1e-6)
        observation_next_norm = \
            (observation_next - state.norm.observation_mean) / (state.norm.observation_std + 1e-6)

        action_norm = (action - state.norm.action_mean) / (state.norm.action_std + 1e-6)
        
        if model == 'D':
            prediction = reward_model(state.params, observation_norm, 
                                      action_norm, observation_next_norm)
        elif model == 'P':
            prediction = reward_model(state.params, observation_norm, 
                                      action_norm, observation_next_norm).loc
        else:
            raise NotImplementedError()
        return prediction
    
    info = tax.reduce(allinfo)
    return state, rmodel_inference, info


In [6]:
train = partial(train_rmodel, loss_fn=loss, update_fn=update,
                seed = 42, batch_size= 256, 
                reward_model = rmodel.apply,
                use_norm = False, 
                max_epochs = 500, validation_size = 0.25,
                early_stopping_patience = 10, alpha_norm = 0.5)

In [7]:
data['reward'] = data['reward'][..., None]

In [8]:
rstate, rmodel_inference, info = train(data, rstate)

100%|█████████▉| 499/500 [00:25<00:00, 19.21it/s, loss/train=0.0021, loss/valid=0.00383] 


In [11]:
from mbrl.algs.rs import trajectory_search, forecast, score, plan

def world(carry, t):
    keys, (env_state, observation), trajectory = carry
    action = trajectory[t]
    env_state_next, observation_next, true_reward, terminal, info = \
        env.step(env_state, action)
    reward = rmodel_inference(None, observation, action, observation_next)
    carry = keys, (env_state_next, observation_next), trajectory
    return carry, {
        "observation": observation,
        "observation_next": observation_next,
        "reward": reward, "action": action, "terminal": 1 - terminal,
        "env_state": env_state, 'env_state_next': env_state_next
    }

def one_step(carry, t):
    key, (env_state, observation)  = carry
    key, subkey = jax.random.split(key)
    action, action_info = plan(subkey, (env_state, observation), forecast_, score_)
    action = action[0]
    env_state_next, observation_next, reward, terminal, info = \
        env.step(env_state, action)
    carry = key, (env_state_next, observation_next )
    return carry, {
        "observation": observation,
        "observation_next": observation_next,
        "reward": reward, "action": action, "terminal": 1 - terminal,
        "env_state": env_state, 'env_state_next': env_state_next,
    }

score_ = jit(partial(score, terminal_reward_fn = None, discount = 0.99))
forecast_ = jit(partial(
    forecast, 
    step_fn=world, 
    horizon=20, 
    action_dim=1, 
    minval=-2., 
    maxval=2.))

In [12]:
%%time
for _ in range(10):
    rng, subrng = jax.random.split(rng)
    env_state, observation = env.reset(subrng)
    init = (rng, (env_state, observation))
    _, out = jax.lax.scan(one_step, init, jnp.arange(200))
    print(jnp.sum(out['reward']))
    action = out['action']
    env_state = out['env_state']
    env_state_next = out['env_state_next']

-440.58087
-242.65468
-122.7583
-246.43565
-118.41893
-388.31924
-119.87218
-0.80356526
-242.38484
-120.13866
CPU times: user 20.9 s, sys: 1.64 s, total: 22.5 s
Wall time: 11.5 s
